In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Lib.module.risk_budgeting import *
import student_mixture as sm
from tqdm import tqdm
from Lib.module.movidas import *

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
np.random.seed(0)

In [11]:
### Import stock price data for model calibration
freq = 'B'
assets = ['JPM UN Equity', 'PFE UN Equity', 'XOM UN Equity']
nb_asset = len(assets)

df_all = pd.concat([pd.read_excel('Data/SP_RC.xlsx',index_col=0, header=[0,1]), pd.read_excel('Data/SP_RC2.xlsx',index_col=0, header=[0,1])])
df_all = df_all.dropna(how='all').dropna(axis=1)
df_all = df_all.pct_change().dropna()
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all[~df_all.index.duplicated(keep='first')]
df = df_all[assets]

X = df.values
print(X.shape)

(3446, 3)


In [12]:
### Fit Student-t mixture (2 components) to chosen stock returns
n_sm = 2
SM_model = sm.StudentMixture(n_components=n_sm, fixed_dofs = True, dofs_init = [2.5,4]).fit(X)

In [13]:
### Print model parameters 
print("==== Number of assets ====")
print(nb_asset)
print("==== Number of mixture components ====")
print(n_sm)
print("==== Weights (probability) parameters ====")
print(SM_model.weights_)
print("==== Degree of freedom parameters ====")
print(SM_model.dofs_)
print("==== Location parameters ====")
print(SM_model.locations_)
print("==== Scale parameters ====")
print(SM_model.scales_)

==== Number of assets ====
3
==== Number of mixture components ====
2
==== Weights (probability) parameters ====
[0.57536895 0.42463105]
==== Degree of freedom parameters ====
[2.5 4. ]
==== Location parameters ====
[[-0.00070383 -0.00097538 -0.00182552]
 [ 0.00160405  0.00209373  0.00204925]]
==== Scale parameters ====
[[[2.45546634e-04 7.09484995e-05 8.74424168e-05]
  [7.09484995e-05 6.90364583e-05 4.56142046e-05]
  [8.74424168e-05 4.56142046e-05 1.04945729e-04]]

 [[9.06453859e-05 1.79115311e-05 4.41578466e-05]
  [1.79115311e-05 1.24689694e-04 1.80013614e-05]
  [4.41578466e-05 1.80013614e-05 1.06182789e-04]]]


In [14]:
### Compute the Risk Budgeting portfolio under the above model

# ERC
budgets = np.ones(nb_asset)/nb_asset 
# Expected Shortfall alpha
alpha = .95 

SM_theta, optim_res = StudentMixtureExpectedShortfall(SM_model).solve_risk_budgeting(budgets, alpha, on_simplex=False, kappa=1, method=None, maxiter=15000)
VaR_port = StudentMixtureExpectedShortfall(SM_model).value_at_risk(SM_theta, alpha)
ES_port = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(SM_theta, alpha)
risk_contribs = SM_theta * StudentMixtureExpectedShortfall(SM_model).expected_shortfall_gradient(SM_theta, alpha)

print('==== Target Risk Bugdets ====')
print(np.round(budgets, 4))
print('==== $y^*$ ====')
print(np.round(optim_res.x, 4))

print('==== Risk Budgeting portfolio ($u^*$) ====')
print(np.round(SM_theta, 4))
print('==== VaR of the portfolio ====')
print(np.round(VaR_port, 4))
print('==== Expected Shortfall of the portfolio ====')
print(np.round(ES_port, 4))
print('==== Risk Contributions ====')
print(np.round(risk_contribs, 5))
print('==== Risk Contributions (normalized) ====')
print(np.round(risk_contribs/sum(risk_contribs), 2))

==== Target Risk Bugdets ====
[0.3333 0.3333 0.3333]
==== $y^*$ ====
[ 6.9959 11.6543  9.8087]
==== Risk Budgeting portfolio ($u^*$) ====
[0.2458 0.4095 0.3447]
==== VaR of the portfolio ====
0.0202
==== Expected Shortfall of the portfolio ====
0.0351
==== Risk Contributions ====
[0.01171 0.01171 0.01171]
==== Risk Contributions (normalized) ====
[0.33 0.33 0.33]


In [15]:
optimal_loss = StudentMixtureExpectedShortfall(SM_model).expected_shortfall(optim_res.x, alpha) - np.dot(budgets, np.log(optim_res.x))
optimal_loss

-1.2280854838694133

### SMD vs. SGD - Error analysis

In [ ]:
np.random.seed(0)

errors_sgd = []
errors_smd = []

for i in tqdm(range(100)):

    n_val=1000000
    X = SM_model.rvs(n_val)
    np.random.shuffle(X)

    alpha = .95
    gamma_sgd = 25
    gamma_smd = 1

    y_sgd = budgets / np.std(X, axis=0)
    xi_sgd = 0
    c_sgd = .65

    y_smd = budgets / np.std(X, axis=0)
    xi_smd = 0
    c_smd = .65
    M = 500

    y_sgd_s = [y_sgd]
    xi_sgd_s = [xi_sgd]
    y_bar_sgd_s = [y_sgd]
    y_bar_sgd_numerator = 0
    y_bar_sgd_denominator = 0

    y_smd_s = [y_smd]
    xi_smd_s = [xi_smd]
    y_bar_smd_s = [y_smd]
    y_bar_smd_numerator = 0
    y_bar_smd_denominator = 0

    error_sgd = []
    error_smd = []

    freq_error = 50

    for k in range(1, n_val):
        x = X[k]
        
        ### SGD
        # gradient
        step_size_sgd = gamma_sgd/k**c_sgd
        indicator_sgd = -np.dot(y_sgd, x) - xi_sgd >= 0
        grad_y_sgd = -x/(1-alpha)*indicator_sgd - budgets/y_sgd
        grad_xi_sgd = 1 - (1 / (1 - alpha)) * indicator_sgd

        #descent
        y_sgd = y_sgd - step_size_sgd*grad_y_sgd*min(min(y_sgd),1)
        y_sgd = np.where(y_sgd <= 0, 1e-04, y_sgd)
        xi_sgd = xi_sgd - step_size_sgd*grad_xi_sgd
        y_bar_sgd_numerator += y_sgd*step_size_sgd
        y_bar_sgd_denominator += step_size_sgd

        ### SMD
        # gradient
        step_size_smd = gamma_smd/k**c_smd
        indicator_smd = -np.dot(y_smd, x) - xi_smd >= 0
        grad_y_smd = -x/(1-alpha)*indicator_smd - budgets/y_smd
        grad_xi_smd = 1 - (1 / (1 - alpha)) * indicator_smd

        y_smd_min = min(min(y_smd),1)
        y_smd = y_smd*np.exp(-step_size_smd*y_smd_min*grad_y_smd)
        xi_smd = xi_smd - step_size_smd*grad_xi_smd
        sum_y_smd = np.sum(y_smd)
        if sum_y_smd>M:
            y_smd = M/sum_y_smd*y_smd
        y_bar_smd_numerator += y_smd*step_size_smd
        y_bar_smd_denominator += step_size_smd

        # y_sgd_s.append(y_sgd)
        # xi_sgd_s.append(xi_sgd)
        # y_bar_sgd_s.append(y_bar_sgd_numerator/y_bar_sgd_denominator)

        # y_smd_s.append(y_smd)
        # xi_smd_s.append(xi_smd)
        # y_bar_smd_s.append(y_bar_smd_numerator/y_bar_smd_denominator)

        if k%freq_error==0:
            error_sgd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_sgd, alpha) - np.dot(budgets, np.log(y_sgd)) - optimal_loss)
            error_smd.append(StudentMixtureExpectedShortfall(SM_model).expected_shortfall(y_smd, alpha) - np.dot(budgets, np.log(y_smd)) - optimal_loss)
    
    errors_sgd.append(error_sgd)
    errors_smd.append(error_smd)

    np.save('Output/sgd_errors_tamed.npy', np.array(errors_sgd))
    np.save('Output/smd_errors_tamed.npy', np.array(errors_smd))

100%|██████████| 100/100 [4:04:58<00:00, 146.99s/it] 
